In [0]:
processName=dbutils.widgets.get('prm_processName')

nextSourceFileDateSql=f"""select nvl(max(PROCESSED_FILE_TABLE_DATE)+1,'2023-01-01') as NEXT_SOURCE_FILE_DATE from pricing_analytics.processrunlogs.delta_lakehouse_process_runs
where PROCESS_NAME = '{processName}' and PROCESS_STATUS='Completed'"""

nextSourceFileDF=spark.sql(nextSourceFileDateSql)



In [0]:
from datetime import datetime





In [0]:
dailyPricingSourceBaseURL="https://retailpricing.blob.core.windows.net/"
dailyPricingSourceFolder="daily-pricing/"
dailyPricingSourceFileDate=datetime.strptime(str(nextSourceFileDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']), '%Y-%m-%d').strftime('%d%m%Y')
dailyPricingSourceFileName=f"PW_MW_DR_{dailyPricingSourceFileDate}.csv"




dailyPricingSinkLayerNAme="bronze"
dailyPricingSinkStorageAccountName="stp1dev"
dailyPricingSinkFolderName="daily-pricing"


In [0]:
import pandas as pd

In [0]:
dailyPricingSourceURL=dailyPricingSourceBaseURL+dailyPricingSourceFolder+dailyPricingSourceFileName


In [0]:
dailyPricingPandasDF=pd.read_csv(dailyPricingSourceURL)


In [0]:
dailyPricingSparkDF=spark.createDataFrame(dailyPricingPandasDF)


In [0]:
from pyspark.sql.functions import *

dailyPricingSinkFolderPath = f"abfss://{dailyPricingSinkLayerNAme}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"

dailyPricingSparkDF.withColumn("source_file_load_date", current_timestamp()) \
                   .write \
                   .format("csv") \
                   .mode("append") \
                   .option("header", "true") \
                   .save(dailyPricingSinkFolderPath)

In [0]:

processFileDate=nextSourceFileDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']
processStatus="Completed"
processInsertSql=f"""insert into pricing_analytics.processrunlogs.DELTA_LAKEHOUSE_PROCESS_RUNS(PROCESS_NAME,PROCESSED_FILE_TABLE_DATE,PROCESS_STATUS) values('{processName}','{processFileDate}','{processStatus}') """

spark.sql(processInsertSql)